# De zenith-afhankelijkheid van een station.

Een HiSPARC-station meet deeltjes in een deeltjeslawine die wordt veroorzaakt door een primair kosmisch deeltje. De deeltjes in de lawine bewegen globaal met de lichtsnelheid. De interactie van het primaire deeltje vindt plaats op een hoogte van tientallen kilometers. De doorsnede van de deeltjeslawine is slechts enkele hectometers. We kunnen we er dus van uitgaan dat deeltjes globaal in een vlak loodrecht op de snelheid van het primaire deeltje bewegen.

In [1]:
import numpy as np
import sapphire
from sapphire import Station
detectors = Station(508).detectors()
for detector in detectors:
    print detector

{u'mpv': 262.0, u'alpha': 135.0, u'beta': -45.0, u'radius': 8.66, u'height': 0.0}
{u'mpv': None, u'alpha': 184.1, u'beta': -45.0, u'radius': 13.22, u'height': 0.0}
{u'mpv': 235.0, u'alpha': 45.0, u'beta': 45.0, u'radius': 5.0, u'height': 0.0}
{u'mpv': 221.0, u'alpha': 225.0, u'beta': 45.0, u'radius': 5.0, u'height': 0.0}


De eigenschappen van een station zijn gedefinieerd met:

- 'mpv' : Most Probable Value. De spanning waarbij de grafiek van het aantal pulsen als functie van de spanning horizontaal is. Zie bijvoorbeeld: [http://data.hisparc.nl/show/stations/3401/2015/11/23/](http://data.hisparc.nl/show/stations/3401/2015/11/23/).
- 'alpha' : De hoek tussen de as gedefinieerd door de GPS-antenne en het midden van de scintillatorplaat en het Noorden. 
- 'beta' : De hoek tussen de lange zijde van de scintillatorplaat en het Noorden.
- 'radius' : De afstand van GPS- antenne tot het midden van de detector.

Een werkblad voor het maken van een stations-plattegrond is te vinden op:

[http://docs.hisparc.nl/infopakket/pdf/station_map.pdf](http://docs.hisparc.nl/infopakket/pdf/station_map.pdf)

De afstand tussen twee detectoren is te bepalen met de cosinusregel. Dit is in een functie te beschrijven:

In [2]:
def afstand(detector_1, detector_2):
    '''
    
    '''
    c = detector_2['radius']
    b = detector_1['radius']
    alpha = np.radians(detector_2['alpha'] - detector_1['alpha'])
    # numpy werkt meet de hoek in radialen.
    return (b ** 2 + c ** 2 - 2 * b * c * np.cos(alpha)) ** .5

De afstand tussen detector 1 (telt als 0) en 4 (telt als 3) is nu te berekenen met:

In [3]:
print afstand(detectors[0], detectors[3])

9.99977999758


In het notebook python_data_retrieval is uitgelegd hoe data op te halen is:

In [4]:
from sapphire import quick_download
data = quick_download(508)

100%|############################################################|Time: 0:00:05


In [5]:
tijden = [10, 11, 12, 13] # Met deze waarden wordt naar de aankomsttijden bij de detectoren 1, 2, 3 en 4 gewezen.
for tijd in tijden:
    print data.root.s508.events[0][tijd]

15.0
-999.0
-999.0
35.0


In [6]:
def zenithoeken(event, detectors):
    ''' 
    De zenithoek is de hoek tussen het golf-front en de horizon, 
    of ook tussen de as naar het zenith en de as van de deeltjeslawine.
    
    parameters
    event: een enkel event uit de opgehaalde data
    detectors: de detectorinformatie uit de API
    
    returns
    een array met de hoek tussen de as door twee detectors en het deeltjes front.
    '''
    c = 0.2997 # in m/ns
    zenith = []
    for i in range(0, 3):
        for j in range(i + 1, 4):
            schuine = afstand(detectors[i], detectors[j])
            overstaande = c * (event[i + 10] - event[j + 10]) / 100
            try:
                angle = np.degrees(np.arcsin(overstaande / schuine))
            except:
                angle = -999
            zenith.append(angle)
    return zenith

In [7]:
event = data.root.s508.events[0]
print zenithoeken(event, detectors)

[17.70567568881863, 17.692147018848907, -0.34344051476589926, 0.0, -18.067119899239444, -18.052589454881673]


In [9]:
selecties = range(0, 10)
for selectie in selecties:
    event = data.root.s508.events[selectie]
    print zenithoeken(event, detectors)

[17.70567568881863, 17.692147018848907, -0.34344051476589926, 0.0, -18.067119899239444, -18.052589454881673]
[0.0, -17.647091611648843, -25.216061588313405, -10.085151972168397, -25.236461797443695, -7.058171681758421]
[-17.70567568881863, 0.0, -17.827381214060242, 10.110339844335856, -0.12888689953425922, -17.826975840906918]
[-0.30073246327933972, 17.647091611648843, 17.647091611648843, 10.261508730688066, 17.976762245897277, 0.0]
[-0.085923198616668431, 0.0, -0.085859646667091408, 0.04959543379722129, 0.0, -0.085857757732672346]
[0.0, -17.827381214060242, -17.647091611648843, -10.185915329754817, -17.660872452606746, 0.17171570826003479]
[17.660583543975601, 17.647091611648843, -0.085859646667091408, 0.0, -17.751069648693221, -17.736810545292407]
[0.0, -17.602047472751252, -48.178036981002393, -10.059966069265851, -48.226470483458286, -26.283100425617246]
[0.0, -17.647091611648843, -17.737213730694243, -10.085151972168397, -17.751069648693221, -0.085857757732672346]
[-17.61550269501

Hierboven zijn de gereconstrueerde hoeken voor de eerste 10 events voor alle combinaties van 2 detectoren in een set van 4 detectoren berekend. De hoeken zijn gedefinieerd als de hoek tussen de as door de detector en het hart van de deeltjeslawine.